In [ ]:
%load_ext sql
%sql sqlite:///aidatabase_experiment.db

In [ ]:
%%sql

CREATE TABLE IF NOT EXISTS Data (
    ID INTEGER PRIMARY KEY,
    ExperimentID TEXT,
    Time REAL,
    Temperature REAL,
    Humidity REAL,
    Gas REAL,
    CO2 REAL
);

CREATE TABLE IF NOT EXISTS Metadata (
    ID INTEGER PRIMARY KEY,
    ExperimentID TEXT,
    NiclaID TEXT,
    MKRID TEXT,
    Location TEXT,
    Date_Start TEXT,
    Duration INTEGER,
    Weather_Description TEXT
);

In [ ]:
import sqlite3
import csv
import json

url_csv = 'url_to_csv'
url = 'url'
filename1 = 'date-file'
filename2 = 'date-file'
filename3 = 'metadata.json'
filename4 = 'metadata2.json'

# Function to insert data from a CSV file into the database
def insert_data_from_csv(csv_filename, cursor, filename):
    '''This function adds all data from CSV file into SQL database'''
    with open(csv_filename, 'r') as f:
        csv_reader = csv.reader(f)
        next(csv_reader)  # Skip the header row if present
        # Loop through rows and insert into the database
        for row in csv_reader:
            cursor.execute('INSERT INTO Data (ExperimentID, Time, Temperature, Humidity, Gas, CO2) VALUES (?, ?, ?, ?, ?, ?)', (filename, row[0], row[1], row[2], row[3], row[4]))

# Function to insert data from JSON file into the database
def insert_data_from_json(json_filename, cursor):
    with open(json_filename, 'r') as f:
        data = json.load(f)  # Load the JSON data
        # Loop through each entry in the JSON file and insert into the database
        for key, entry in data.items():
            # Assuming the JSON structure is a list of dictionaries with the same fields as the database columns
            cursor.execute('INSERT INTO Metadata (ExperimentID, NiclaID, MKRID, Location, Date_Start, Duration, Weather_Description) VALUES (?, ?, ?, ?, ?, ?, ?)',
                           (key, entry['NiclaID'], entry['MKRID'], entry['Location'], entry['Date_Start'], float(entry['Duration']), entry['Weather_Description']))

# Connect to SQLite database
conn = sqlite3.connect('aidatabase_experiment.db')
cursor = conn.cursor()

# Insert data from the first file
insert_data_from_csv(f"{url_csv}{filename1}.csv", cursor, filename1)

# Insert data from the second file
insert_data_from_csv(f"{url_csv}{filename2}.csv", cursor, filename2)

# Insert data from json file
insert_data_from_json(f"{url}{filename3}", cursor)

# Insert data from second json file
insert_data_from_json(f"{url}{filename4}", cursor)

# Commit the changes to the database
conn.commit()

# Close the connection
conn.close()

In [ ]:
%%sql
SELECT * FROM Metadata;

In [ ]:
%%sql
SELECT * FROM Data;